In [39]:
import re
from datetime import datetime
from pathlib import Path
from typing import Any

import numpy as np
import polars as pl
import xarray as xr

gnd_path = Path.cwd() / "output/C1gnd.out"

with gnd_path.open("r") as f:
    lines = f.readlines()


In [40]:
def parse_geometry_line(line: str) -> dict[str, Any] | None:
    """
    Parse a single line from the geometry section.

    Args:
        line (str): Line containing atom geometry data

    Returns
    -------
        dict or None: Dictionary with atom information or None if parsing failed
    """
    # Don't try to parse header or separator lines
    if "----" in line or ("Atom" in line and "x" in line):
        return None

    # Simple but robust pattern matching - split by whitespace
    parts = line.strip().split()

    # Check if we have enough parts to make a valid atom entry
    if len(parts) >= 8:  # At least 8 elements for a geometry line
        try:
            # Extract the index and atom name considering the format "1) C01"
            idx_str = parts[0]
            atom_str = parts[1]

            # If the first part has a trailing ")", it contains the index
            if idx_str.endswith(")"):
                _ = idx_str.rstrip(")")
            else:
                # Handle case where index and atom are merged like "1)C01"
                match = re.match(r"(\d+)\)(.*)", idx_str)
                if match:
                    atom_str = match.group(2)

            return {
                "atom": atom_str,
                "x": float(parts[2]) if len(parts) > 2 else 0.0,
                "y": float(parts[3]) if len(parts) > 3 else 0.0,
                "z": float(parts[4]) if len(parts) > 4 else 0.0,
                "q": float(parts[5]) if len(parts) > 5 else 0.0,
                "nuc": int(parts[6]) if len(parts) > 6 else 0,
                "mass": float(parts[7]) if len(parts) > 7 else 0.0,
                "neq": parts[8] if len(parts) > 8 else "",
                "grid": int(parts[9]) if len(parts) > 9 else 0,
                "grp": int(parts[10]) if len(parts) > 10 else 0,
            }
        except (ValueError, IndexError):
            # Log diagnostic info if needed
            # print(f"Failed to parse line: {line}")
            return None

    return None


def extract_section(
    content: list[str], start_pattern: str, end_pattern: str
) -> list[str]:
    """
    Extract a section from the file content between start and end patterns.

    Args:
        content (list): List of lines from the file
        start_pattern (str): Regex pattern to match the start of the section
        end_pattern (str): Regex pattern to match the end of the section

    Returns
    -------
        list: Lines of the extracted section
    """
    section = []
    in_section = False
    for line in content:
        # Check if we're at the start of the section
        if not in_section and re.search(start_pattern, line):
            in_section = True
            section.append(line)
        # If we're already in the section
        elif in_section:
            section.append(line)
            # Check if we've reached the end of the section
            if re.search(end_pattern, line):
                break

    return section

In [41]:
def extract_geometry(lines: list[str]) -> pl.DataFrame:
    """
    Extract the geometry section from the GND file.
    """
    geometry_section = extract_section(
        lines,
        start_pattern=r"INPUT GEOMETRY \(input file\)",
        end_pattern=r"Smallest atom distance.*?=.*?\d+\.\d+",
    )
    geometry = pl.DataFrame(
        [
            parse_geometry_line(line)
            for line in geometry_section
            if line.strip() and parse_geometry_line(line) is not None
        ]
    )
    return geometry


geometry = extract_geometry(lines)
geometry

atom,x,y,z,q,nuc,mass,neq,grid,grp
str,f64,f64,f64,f64,i64,f64,str,i64,i64
"""C01""",1.096564,-2.795541,-0.126884,6.0,6,12.011,"""1/""",1,32
"""C02""",0.650113,-4.188295,-0.15538,4.0,6,12.011,"""2/""",1,32
"""C03""",1.330451,-5.393227,-0.302897,4.0,6,12.011,"""3/""",1,32
"""C04""",0.576055,-6.558919,-0.305203,4.0,6,12.011,"""4/""",1,32
"""C05""",-0.812707,-6.525957,-0.159831,4.0,6,12.011,"""5/""",1,32
…,…,…,…,…,…,…,…,…,…
"""N05""",-2.37713,2.429006,0.208542,7.0,7,14.007,"""53/""",1,32
"""N06""",0.0,1.982701,0.0,7.0,7,14.007,"""54/""",1,32
"""N07""",2.38824,2.356362,-0.214891,7.0,7,14.007,"""55/""",1,32


In [42]:
# pass geometry into an xarray dataset

geometry_ds = xr.Dataset(
    {
        "x": (("atom"), geometry["x"].to_numpy()),
        "y": (("atom"), geometry["y"].to_numpy()),
        "z": (("atom"), geometry["z"].to_numpy()),
        "q": (("atom"), geometry["q"].to_numpy()),
        "nuc": (("atom"), geometry["nuc"].to_numpy()),
        "mass": (("atom"), geometry["mass"].to_numpy()),
        "neq": (("atom"), geometry["neq"].to_numpy()),
        "grid": (("atom"), geometry["grid"].to_numpy()),
        "grp": (("atom"), geometry["grp"].to_numpy()),
    },
    coords={
        "atom": geometry["atom"].to_numpy(),
    },
)
geometry_ds.to_netcdf(
    Path.cwd() / "output/geometry.nc",
    mode="w",
    engine="h5netcdf",
)
geometry_ds

<xarray.Dataset> Size: 5kB
Dimensions:  (atom: 57)
Coordinates:
  * atom     (atom) object 456B 'C01' 'C02' 'C03' 'C04' ... 'N07' 'N08' 'Zn01'
Data variables:
    x        (atom) float64 456B 1.097 0.6501 1.33 0.5761 ... 2.388 1.973 0.0
    y        (atom) float64 456B -2.796 -4.188 -5.393 ... 2.356 -0.03092 0.0
    z        (atom) float64 456B -0.1269 -0.1554 -0.3029 ... -0.2149 -0.1647 0.0
    q        (atom) float64 456B 6.0 4.0 4.0 4.0 4.0 ... 7.0 7.0 7.0 7.0 30.0
    nuc      (atom) int64 456B 6 6 6 6 6 6 6 6 6 6 6 ... 1 1 7 7 7 7 7 7 7 7 30
    mass     (atom) float64 456B 12.01 12.01 12.01 12.01 ... 14.01 14.01 65.37
    neq      (atom) object 456B '1/' '2/' '3/' '4/' ... '54/' '55/' '56/' '57/'
    grid     (atom) int64 456B 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
    grp      (atom) int64 456B 32 32 32 32 32 32 32 32 ... 32 32 32 32 32 32 32

In [43]:
def extract_basis_set(line):
    """
    Extract the basis set information from a line.

    Parameters
    ----------
    line : str
        A line from the basis set section of the GND file.

    Returns
    -------
    dict or None
        A dictionary with the basis set information or None if parsing failed.
    """
    # ensure the line is not the header or separator line
    if "I" in line or "II" in line or "III" in line:
        return None
    # parse the line by atom, and basis set
    parts = line.split()
    if len(parts) < 2:
        return None
    return {
        "atom": parts[1],
        "basis": " ".join(parts[3:]),
    }


def extract_basis_sets(lines: list[str]) -> pl.DataFrame:
    """
    Extract the basis sets from the GND file.

    Parameters
    ----------
    lines : list[str]
        The lines from the GND file.

    Returns
    -------
    pl.DataFrame
        A DataFrame containing the basis set information.
    """
    # Extract different basis set sections
    section_configs = [
        {
            "name": "aux",
            "start": r"I\)  AUXILIARY BASIS SETS",
            "end": r"II\)  ORBITAL BASIS SETS",
            "suffix": "",
        },
        {
            "name": "orbital",
            "start": r"II\)  ORBITAL BASIS SETS",
            "end": r" BASIS DIMENSIONS",
            "suffix": "_orbital",
        },
        {
            "name": "model_core",
            "start": r"III\)  MODEL POTENTIALS",
            "end": r" \(NEW\) SYMMETRIZATION INFORMATION",
            "suffix": "_model_core",
        },
    ]

    # Extract and process each section
    dfs = {}
    for config in section_configs:
        section = extract_section(
            lines,
            start_pattern=config["start"],
            end_pattern=config["end"],
        )

        # Filter and parse data
        data = [
            entry
            for line in section
            if line.strip() and (entry := extract_basis_set(line)) is not None
        ]

        # Create DataFrame or empty placeholder
        dfs[config["name"]] = (
            pl.DataFrame(data) if data else pl.DataFrame({"atom": [], "basis": []})
        )

    # Combine DataFrames using left joins to preserve all rows from orbital basis
    basis_sets = (
        dfs["orbital"]
        .join(dfs["aux"], on="atom", how="left", suffix=" auxiliary")
        .join(
            dfs["model_core"],
            on="atom",
            how="left",
            suffix=" model core potential",
        )
    )

    return basis_sets


basis = extract_basis_sets(lines)

basis_ds = xr.Dataset(
    {
        "orbital_basis": (("atom"), basis["basis"].to_numpy()),
        "auxiliary_basis": (("atom"), basis["basis auxiliary"].to_numpy()),
        "model_core_basis": (
            ("atom"),
            basis["basis model core potential"].to_numpy(),
        ),
    },
    coords={
        "atom": basis["atom"].to_numpy(),
    },
)
basis_ds

<xarray.Dataset> Size: 2kB
Dimensions:           (atom: 48)
Coordinates:
  * atom              (atom) object 384B 'C01' 'C02' 'C03' ... 'H14' 'H15' 'H16'
Data variables:
    orbital_basis     (atom) object 384B 'O-CARBON iii_iglo' ... 'O-HYDROGEN ...
    auxiliary_basis   (atom) object 384B 'A-CARBON (5,2;5,2)' ... 'A-HYDROGEN...
    model_core_basis  (atom) object 384B None 'P-CARBON(+4) (3,1:8,0)' ... None

In [44]:
def extract_energy(line: str) -> dict[str, Any] | None:
    """
    Extract energy information from a line.

    Args:
        line (str): Line containing energy data
        pattern (str): Regex pattern to match the energy data

    Returns
    -------
        dict or None: Dictionary with energy information or None if parsing failed
    """
    if "<Rho/" in line:
        return None
    # Energies have the pattern
    # \sKind of energy (H) = value
    pattern = r"([\w\s-]+?)\s*\((\w+)\)\s*=\s*([+-]?\d+\.\d+)"
    match = re.search(pattern, line)
    if match:
        energy_type = match.group(1).strip()  # Strip whitespace from the type
        energy_value = float(match.group(3)) * 27.2114  # convert from Hartree to eV
        return {
            "type": energy_type,
            "value": energy_value,
        }
    return None


def extract_energy_section(lines: list[str]) -> list[dict[str, Any]]:
    """
    Extract energy information from the energy section.

    Args:
        lines (list): Lines from the energy section

    Returns
    -------
        list: List of dictionaries with energy information
    """
    energy_section = extract_section(
        lines,
        start_pattern=r" FINAL ENERGY",
        end_pattern=r" Decomposition of",
    )
    return [
        entry
        for line in energy_section
        if line.strip() and (entry := extract_energy(line)) is not None
    ]


ground_energy_section = extract_energy_section(lines)
# Create a dataset with each energy type as a separate variable
ground_energy_ds = xr.Dataset(
    {
        entry["type"].replace(" ", "_").lower(): ([], entry["value"])
        for entry in ground_energy_section
    },
)
ground_energy_ds

<xarray.Dataset> Size: 64B
Dimensions:                   ()
Data variables:
    total_energy              float64 8B -6.636e+04
    nuc-nuc_energy            float64 8B 8.772e+04
    el-nuc_energy             float64 8B -3.334e+05
    kinetic_energy            float64 8B 6.595e+04
    coulomb_energy            float64 8B 1.186e+05
    ex-cor_energy             float64 8B -5.242e+03
    total_exchange_energy     float64 8B -4.996e+03
    total_correlation_energy  float64 8B -245.9

In [45]:
def extract_orbital_energies(lines: list[str]) -> pl.DataFrame:
    """
    Extract orbital energy information from the GND file.

    Parameters
    ----------
    lines : list[str]
        The lines from the GND file.

    Returns
    -------
    dict
        A dictionary containing the orbital energy information for alpha and beta spins.
    """
    orbital_section = extract_section(
        lines,
        start_pattern=r"ORBITAL ENERGIES \(ALL VIRTUALS INCLUDED\)",
        end_pattern=r" LISTING OF SPIN ALPHA ORBITALS ONLY",
    )

    # Skip header lines (2 lines for header, 1 for column names)
    data_lines = orbital_section[3:]

    # Initialize lists to store the parsed data
    orbitals = []
    alpha_occupations = []
    alpha_energies = []
    alpha_symmetries = []
    alpha_positions = []
    beta_occupations = []
    beta_energies = []
    beta_symmetries = []
    beta_positions = []

    # Parse each data line
    for line in data_lines:
        if not line.strip():  # Skip empty lines
            continue

        parts = line.split()
        if len(parts) >= 11:  # Check if we have enough parts for a complete line
            try:
                orbital_num = int(parts[0])
                orbitals.append(orbital_num)

                # Alpha spin data
                alpha_occupations.append(float(parts[1]))
                alpha_energies.append(float(parts[2]))
                alpha_symmetries.append(parts[3])
                alpha_positions.append(int(parts[5].strip("()")))

                # Beta spin data
                beta_occupations.append(float(parts[6]))
                beta_energies.append(float(parts[7]))
                beta_symmetries.append(parts[8])
                beta_positions.append(int(parts[10].strip("()")))
            except (ValueError, IndexError):
                # Skip lines that don't match the expected format
                continue

    return pl.DataFrame(
        {
            "orbital": orbitals,
            "alpha_occupation": alpha_occupations,
            "alpha_energy": alpha_energies,
            "alpha_symmetry": alpha_symmetries,
            "alpha_position": alpha_positions,
            "beta_occupation": beta_occupations,
            "beta_energy": beta_energies,
            "beta_symmetry": beta_symmetries,
            "beta_position": beta_positions,
        }
    )


gnd_spin_section = extract_orbital_energies(lines)
# get the lumo and homo orbital numbers where occupation first = 0
homo_orbital = next(
    (i for i, occ in enumerate(gnd_spin_section["alpha_occupation"]) if occ == 0), None
)
homo_energy = (
    gnd_spin_section["alpha_energy"].to_numpy()[homo_orbital]
    if homo_orbital is not None
    else None
)
lumo_orbital = homo_orbital + 1 if homo_orbital is not None else None
lumo_energy = (
    gnd_spin_section["alpha_energy"].to_numpy()[lumo_orbital]
    if lumo_orbital is not None
    else None
)

# Create a dataset with each energy type as a separate variable
gnd_spin_ds = xr.Dataset(
    {
        "homo": homo_orbital,  # Store the orbital number as a scalar
        "lumo": lumo_orbital,  # Store the orbital number as a scalar
        "homo_energy": homo_energy,  # Also store the energy value
        "lumo_energy": lumo_energy,  # Also store the energy value
        "orbital": (("orbital"), gnd_spin_section["orbital"].to_numpy()),
        "alpha_occupation": (
            ("orbital"),
            gnd_spin_section["alpha_occupation"].to_numpy(),
        ),
        "alpha_energy": (("orbital"), gnd_spin_section["alpha_energy"].to_numpy()),
        "alpha_symmetry": (
            ("orbital"),
            gnd_spin_section["alpha_symmetry"].to_numpy(),
        ),
        "alpha_position": (
            ("orbital"),
            gnd_spin_section["alpha_position"].to_numpy(),
        ),
        "beta_occupation": (
            ("orbital"),
            gnd_spin_section["beta_occupation"].to_numpy(),
        ),
        "beta_energy": (("orbital"), gnd_spin_section["beta_energy"].to_numpy()),
        "beta_symmetry": (("orbital"), gnd_spin_section["beta_symmetry"].to_numpy()),
        "beta_position": (("orbital"), gnd_spin_section["beta_position"].to_numpy()),
    },
    coords={
        "atom": geometry["atom"].to_numpy(),
    },
)
gnd_spin_ds

<xarray.Dataset> Size: 53kB
Dimensions:           (orbital: 727, atom: 57)
Coordinates:
  * orbital           (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
  * atom              (atom) object 456B 'C01' 'C02' 'C03' ... 'N08' 'Zn01'
Data variables:
    homo              int64 8B 116
    lumo              int64 8B 117
    homo_energy       float64 8B -3.399
    lumo_energy       float64 8B -3.365
    alpha_occupation  (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
    alpha_energy      (orbital) float64 6kB -9.408e+03 -1.132e+03 ... 3.612e+03
    alpha_symmetry    (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
    alpha_position    (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
    beta_occupation   (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
    beta_energy       (orbital) float64 6kB -9.408e+03 -1.132e+03 ... 3.612e+03
    beta_symmetry     (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
    beta_position     (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727

In [64]:
def read_stobe_ground(
    file: str | Path,
    author: str | None = None,
    comment: str | None = None,
    save_checkpoint=True,
) -> xr.DataTree:
    """
    Read the GND file and extract relevant data into an xarray DataTree.

    This function extracts geometry, basis sets, energy, and spin information
    from the GND file and organizes it into a structured xarray DataTree.


    Parameters
    ----------
    file : str or Path
        The path to the GND file.

    author : str
        The name of the author.

    comment : str
        A comment regarding the data extraction.

    save_checkpoint : bool, optional
        Whether to save a checkpoint of the extraction process (default is True).

    Returns
    -------
    xr.Dataset
        A dataset containing the extracted data.

    Details
    -------
    This loads the GND file, extracts the geometry, basis sets, energy, and occupation.
    This creates three groups of data in the DataTree:
    - GEOMETRY: Contains the geometry information of the atoms.
    - BASIS: Contains the basis set information for the atoms.
    - GND: Contains the ground state energy and spin information.
        - energy: Contains the ground state energy information.
        - spin: Contains the spin occupation information.

    Example
    -------
    >>> dt = read_stobe_ground(
    ...     "path/to/gnd/file.gnd", author="John Doe", comment="Test extraction"
    ... )
    >>> print(dt)
    <xarray.DataTree>
    <GEOMETRY>: ...
    <BASIS>: ...
    <GND>: <energy>: ... <spin>: ...
    """
    file = Path(file)
    with file.open("r") as f:
        lines = f.readlines()
    geometry_section = extract_geometry(lines)
    basis_section = extract_basis_sets(lines)
    ground_energy_section = extract_energy_section(lines)
    gnd_spin_section = extract_orbital_energies(lines)
    #  create a dataset with each energy type as a separate variable
    geometry_ds = xr.Dataset(
        {
            "x": (("atom"), geometry_section["x"].to_numpy()),
            "y": (("atom"), geometry_section["y"].to_numpy()),
            "z": (("atom"), geometry_section["z"].to_numpy()),
            "q": (("atom"), geometry_section["q"].to_numpy()),
            "nuc": (("atom"), geometry_section["nuc"].to_numpy()),
            "mass": (("atom"), geometry_section["mass"].to_numpy()),
            "neq": (("atom"), geometry_section["neq"].to_numpy()),
            "grid": (("atom"), geometry_section["grid"].to_numpy()),
            "grp": (("atom"), geometry_section["grp"].to_numpy()),
        },
        coords={
            "atom": geometry_section["atom"].to_numpy(),
        },
    )
    basis_ds = xr.Dataset(
        {
            "orbital_basis": (("atom"), basis_section["basis"].to_numpy()),
            "auxiliary_basis": (("atom"), basis_section["basis auxiliary"].to_numpy()),
            "model_core_basis": (
                ("atom"),
                basis_section["basis model core potential"].to_numpy(),
            ),
        },
        coords={
            "atom": basis_section["atom"].to_numpy(),
        },
    )
    energy_ds = xr.Dataset(
        {
            entry["type"].replace(" ", "_").lower(): ([], entry["value"])
            for entry in ground_energy_section
        },
    )
    # extract molecular orbitals
    orbital_section = extract_section(
        lines,
        start_pattern=r" Alpha occupation:",
        end_pattern=r" Beta occupation:",
    )
    # get the alpha occupation numbers for the HOMO

    homo_orbital = int(orbital_section[0].split(":")[1].strip())
    lumo_orbital = homo_orbital + 1
    homo_energy = (
        gnd_spin_section["alpha_energy"].to_numpy()[homo_orbital]
        if homo_orbital is not None
        else None
    )
    lumo_energy = (
        gnd_spin_section["alpha_energy"].to_numpy()[lumo_orbital]
        if lumo_orbital is not None
        else None
    )
    energy_ds["lumo-energy"] = lumo_energy
    energy_ds["homo-energy"] = homo_energy
    # search for a core hole with .5 occupation, if it dow not exit then don't add it
    # do the dextract_basis_set
    core_hole_orbital = gnd_spin_section.filter(pl.col("alpha_occupation").eq(0.5))[
        "orbital"
    ]

    spin_ds = xr.Dataset(
        {
            "homo": homo_orbital,
            "lumo": lumo_orbital,
            "alpha_occupation": (
                ("orbital"),
                gnd_spin_section["alpha_occupation"].to_numpy(),
            ),
            "alpha_energy": (("orbital"), gnd_spin_section["alpha_energy"].to_numpy()),
            "alpha_symmetry": (
                ("orbital"),
                gnd_spin_section["alpha_symmetry"].to_numpy(),
            ),
            "alpha_position": (
                ("orbital"),
                gnd_spin_section["alpha_position"].to_numpy(),
            ),
            "beta_occupation": (
                ("orbital"),
                gnd_spin_section["beta_occupation"].to_numpy(),
            ),
            "beta_energy": (("orbital"), gnd_spin_section["beta_energy"].to_numpy()),
            "beta_symmetry": (
                ("orbital"),
                gnd_spin_section["beta_symmetry"].to_numpy(),
            ),
            "beta_position": (
                ("orbital"),
                gnd_spin_section["beta_position"].to_numpy(),
            ),
        },
        coords={
            "orbital": gnd_spin_section["orbital"].to_numpy(),
        },
    )
    if not core_hole_orbital.is_empty():
        spin_ds["core-hole"] = core_hole_orbital.to_numpy()[0]
        energy_ds["core-hole-energy"] = gnd_spin_section["alpha_energy"].to_numpy()[
            core_hole_orbital.to_numpy()[0]
        ]
    # combine to a single datatree
    # combine to a single datatree
    dt = xr.DataTree()
    gnd = xr.DataTree()
    dt["GEOMETRY"] = geometry_ds
    dt["BASIS"] = basis_ds
    gnd["energy"] = energy_ds
    gnd["spin"] = spin_ds
    dt["GND"] = gnd

    # add metadata
    dt.attrs["date"] = datetime.now().strftime("%Y-%m-%d")
    dt.attrs["source"] = str(file)
    dt.attrs["description"] = "GND file extracted data"
    dt.attrs["version"] = "1.0"
    dt.attrs["additional_info"] = "This dataset contains extracted data from GND files."

    if comment:
        dt.attrs["comment"] = comment
    if author:
        dt.attrs["author"] = author

    if save_checkpoint:
        dt.to_netcdf(
            Path(file).parent / "gnd.nc",
            mode="w",
            engine="h5netcdf",
        )
    return dt


gnd_section = read_stobe_ground(gnd_path)
gnd_section

<xarray.DataTree>
Group: /
│   Attributes:
│       date:             2025-04-14
│       source:           c:\Users\hduva\.projects\dft-learn\output\C1gnd.out
│       description:      GND file extracted data
│       version:          1.0
│       additional_info:  This dataset contains extracted data from GND files.
├── Group: /GEOMETRY
│       Dimensions:  (atom: 57)
│       Coordinates:
│         * atom     (atom) object 456B 'C01' 'C02' 'C03' 'C04' ... 'N07' 'N08' 'Zn01'
│       Data variables:
│           x        (atom) float64 456B 1.097 0.6501 1.33 0.5761 ... 2.388 1.973 0.0
│           y        (atom) float64 456B -2.796 -4.188 -5.393 ... 2.356 -0.03092 0.0
│           z        (atom) float64 456B -0.1269 -0.1554 -0.3029 ... -0.2149 -0.1647 0.0
│           q        (atom) float64 456B 6.0 4.0 4.0 4.0 4.0 ... 7.0 7.0 7.0 7.0 30.0
│           nuc      (atom) int64 456B 6 6 6 6 6 6 6 6 6 6 6 ... 1 1 7 7 7 7 7 7 7 7 30
│           mass     (atom) float64 456B 12.01 12.01 12.01 12.01 ... 14.01 14.01 65.37
│           neq      (atom) object 456B '1/' '2/' '3/' '4/' ... '54/' '55/' '56/' '57/'
│           grid     (atom) int64 456B 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
│           grp      (atom) int64 456B 32 32 32 32 32 32 32 32 ... 32 32 32 32 32 32 32
├── Group: /BASIS
│       Dimensions:           (atom: 48)
│       Coordinates:
│         * atom              (atom) object 384B 'C01' 'C02' 'C03' ... 'H14' 'H15' 'H16'
│       Data variables:
│           orbital_basis     (atom) object 384B 'O-CARBON iii_iglo' ... 'O-HYDROGEN ...
│           auxiliary_basis   (atom) object 384B 'A-CARBON (5,2;5,2)' ... 'A-HYDROGEN...
│           model_core_basis  (atom) object 384B None 'P-CARBON(+4) (3,1:8,0)' ... None
└── Group: /GND
    ├── Group: /GND/energy
    │       Dimensions:                   ()
    │       Data variables:
    │           total_energy              float64 8B -6.636e+04
    │           nuc-nuc_energy            float64 8B 8.772e+04
    │           el-nuc_energy             float64 8B -3.334e+05
    │           kinetic_energy            float64 8B 6.595e+04
    │           coulomb_energy            float64 8B 1.186e+05
    │           ex-cor_energy             float64 8B -5.242e+03
    │           total_exchange_energy     float64 8B -4.996e+03
    │           total_correlation_energy  float64 8B -245.9
    │           lumo-energy               float64 8B -3.365
    │           homo-energy               float64 8B -3.399
    └── Group: /GND/spin
            Dimensions:           (orbital: 727)
            Coordinates:
              * orbital           (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
            Data variables:
                homo              int64 8B 116
                lumo              int64 8B 117
                alpha_occupation  (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
                alpha_energy      (orbital) float64 6kB -9.408e+03 -1.132e+03 ... 3.612e+03
                alpha_symmetry    (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
                alpha_position    (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
                beta_occupation   (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
                beta_energy       (orbital) float64 6kB -9.408e+03 -1.132e+03 ... 3.612e+03
                beta_symmetry     (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
                beta_position     (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727

In [65]:
def read_stobe_excited(
    file: str | Path,
    author: str | None = None,
    comment: str | None = None,
    *,
    save_checkpoint: bool = True,
) -> xr.DataTree:
    """
    Read the EXC file and extract relevant data into an xarray DataTree.

    This function extracts geometry, basis sets, energy, and spin information
    from the EXC file and organizes it into a structured xarray DataTree.


    Parameters
    ----------
    file : str or Path
        The path to the EXC file.

    author : str
        The name of the author.

    comment : str
        A comment regarding the data extraction.

    save_checkpoint : bool, optional
        Whether to save a checkpoint of the extraction process (default is True).

    Returns
    -------
    xr.Dataset
        A dataset containing the extracted data.

    Details
    -------
    This loads the EXC file, extracts the geometry, basis sets, energy, and occupation.
    This creates three groups of data in the DataTree:
    - GEOMETRY: Contains the geometry information of the atoms.
    - BASIS: Contains the basis set information for the atoms.
    - GND: Contains the ground state energy and spin information.
        - energy: Contains the ground state energy information.
        - spin: Contains the spin occupation information.

    Example
    -------
    >>> dt = read_stobe_ground(
    ...     "path/to/gnd/file.gnd", author="John Doe", comment="Test extraction"
    ... )
    >>> print(dt)
    <xarray.DataTree>
    <GEOMETRY>: ...
    <BASIS>: ...
    <EXC>: <energy>: ... <spin>: ...
    """
    bad_name_data_tree = read_stobe_ground(file, author, comment, save_checkpoint=False)
    # rename for excited state
    exc = bad_name_data_tree["GND"].copy()
    exc.name = "EXC"
    del bad_name_data_tree["GND"]
    bad_name_data_tree["EXC"] = exc

    # up

    bad_name_data_tree.attrs["description"] = "EXC file extracted data"

    if save_checkpoint:
        bad_name_data_tree.to_netcdf(
            Path(file).parent / "exc.nc",
            mode="w",
            engine="h5netcdf",
        )
    return bad_name_data_tree


exc_path = Path.cwd() / "output/C1exc.out"
exc_section = read_stobe_excited(exc_path, "test", "test")
exc_section

<xarray.DataTree>
Group: /
│   Attributes:
│       date:             2025-04-14
│       source:           c:\Users\hduva\.projects\dft-learn\output\C1exc.out
│       description:      EXC file extracted data
│       version:          1.0
│       additional_info:  This dataset contains extracted data from GND files.
│       comment:          test
│       author:           test
├── Group: /GEOMETRY
│       Dimensions:  (atom: 57)
│       Coordinates:
│         * atom     (atom) object 456B 'C01' 'C02' 'C03' 'C04' ... 'N07' 'N08' 'Zn01'
│       Data variables:
│           x        (atom) float64 456B 1.097 0.6501 1.33 0.5761 ... 2.388 1.973 0.0
│           y        (atom) float64 456B -2.796 -4.188 -5.393 ... 2.356 -0.03092 0.0
│           z        (atom) float64 456B -0.1269 -0.1554 -0.3029 ... -0.2149 -0.1647 0.0
│           q        (atom) float64 456B 6.0 4.0 4.0 4.0 4.0 ... 7.0 7.0 7.0 7.0 30.0
│           nuc      (atom) int64 456B 6 6 6 6 6 6 6 6 6 6 6 ... 1 1 7 7 7 7 7 7 7 7 30
│           mass     (atom) float64 456B 12.01 12.01 12.01 12.01 ... 14.01 14.01 65.37
│           neq      (atom) object 456B '1/' '2/' '3/' '4/' ... '54/' '55/' '56/' '57/'
│           grid     (atom) int64 456B 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
│           grp      (atom) int64 456B 32 32 32 32 32 32 32 32 ... 32 32 32 32 32 32 32
├── Group: /BASIS
│       Dimensions:           (atom: 48)
│       Coordinates:
│         * atom              (atom) object 384B 'C01' 'C02' 'C03' ... 'H14' 'H15' 'H16'
│       Data variables:
│           orbital_basis     (atom) object 384B 'O-CARBON iii_iglo' ... 'O-HYDROGEN ...
│           auxiliary_basis   (atom) object 384B 'A-CARBON (5,2;5,2)' ... 'A-HYDROGEN...
│           model_core_basis  (atom) object 384B None 'P-CARBON(+4) (3,1:8,0)' ... None
└── Group: /EXC
    ├── Group: /EXC/energy
    │       Dimensions:                   ()
    │       Data variables:
    │           total_energy              float64 8B -6.607e+04
    │           nuc-nuc_energy            float64 8B 8.772e+04
    │           el-nuc_energy             float64 8B -3.327e+05
    │           kinetic_energy            float64 8B 6.566e+04
    │           coulomb_energy            float64 8B 1.185e+05
    │           ex-cor_energy             float64 8B -5.215e+03
    │           total_exchange_energy     float64 8B -4.969e+03
    │           total_correlation_energy  float64 8B -246.0
    │           lumo-energy               float64 8B -2.164
    │           homo-energy               float64 8B -3.464
    └── Group: /EXC/spin
            Dimensions:           (orbital: 727)
            Coordinates:
              * orbital           (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
            Data variables:
                homo              int64 8B 117
                lumo              int64 8B 118
                alpha_occupation  (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
                alpha_energy      (orbital) float64 6kB -9.408e+03 -1.132e+03 ... 3.582e+03
                alpha_symmetry    (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
                alpha_position    (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
                beta_occupation   (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
                beta_energy       (orbital) float64 6kB -9.408e+03 -1.132e+03 ... 3.54e+03
                beta_symmetry     (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
                beta_position     (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727

In [66]:
def process_transition_line(line):
    """
    Process a line from the transition section of the EXC file.
    """
    parts = line.split()
    if len(parts) < 9:
        return None
    return {
        "energy": float(parts[2]),
        "oscillator_strength": float(parts[3]),
        "oslx": float(parts[4]),
        "osly": float(parts[5]),
        "oslz": float(parts[6]),
        "osc_r2": float(parts[7]),
        "<r2>": float(parts[8]),
    }


def extract_transitions(lines: list[str]) -> pl.DataFrame:
    """
    Extract the transitions from the EXC file.

    Parameters
    ----------
    lines : list[str]
        The lines from the EXC file.

    Returns
    -------
    pl.DataFrame
        A DataFrame containing the transition information.
    """
    transition_section = extract_section(
        lines,
        start_pattern=r"\s+E \(eV\)\s+OSCL\s+oslx\s+osly\s+oslz\s+osc\(r2\)\s+<r2> ",
        end_pattern=r"\s+SUM\s+[\d.]+\s+[\d.]+",
    )
    transitions = [
        process_transition_line(line)
        for line in transition_section
        if line.strip() and process_transition_line(line) is not None
    ]
    return pl.DataFrame(transitions)


def read_stobe_tp(
    file: str | Path,
    author: str | None = None,
    comment: str | None = None,
    *,
    save_checkpoint: bool = True,
) -> xr.DataTree:
    """
    Read the EXC file and extract relevant data into an xarray DataTree.

    This function extracts geometry, basis sets, energy, and spin information
    from the EXC file and organizes it into a structured xarray DataTree.


    Parameters
    ----------
    file : str or Path
        The path to the EXC file.

    author : str
        The name of the author.

    comment : str
        A comment regarding the data extraction.

    save_checkpoint : bool, optional
        Whether to save a checkpoint of the extraction process (default is True).

    Returns
    -------
    xr.Dataset
        A dataset containing the extracted data.

    Details
    -------
    This loads the EXC file, extracts the geometry, basis sets, energy, and occupation.
    This creates three groups of data in the DataTree:
    - GEOMETRY: Contains the geometry information of the atoms.
    - BASIS: Contains the basis set information for the atoms.
    - GND: Contains the ground state energy and spin information.
        - energy: Contains the ground state energy information.
        - spin: Contains the spin occupation information.
        -

    Example
    -------
    >>> dt = read_stobe_tp(
    ...     "path/to/tp/file.tp", author="John Doe", comment="Test extraction"
    ... )
    >>> print(dt)
    <xarray.DataTree>
    <GEOMETRY>: ...
    <BASIS>: ...
    <TP>: <energy>: ... <spin>: ...<transitions>: ...
    """
    bad_name_data_tree = read_stobe_excited(
        file, author, comment, save_checkpoint=False
    )
    # rename for excited state
    tp = bad_name_data_tree["EXC"].copy()
    tp.name = "TP"
    del bad_name_data_tree["EXC"]
    # add transitions
    # First, read the file content
    with Path(file).open("r") as f:
        lines = f.readlines()

    # extract the core hole information
    core_hole_section = extract_section(
        lines,
        start_pattern=r"Orbital energy core hole",
        end_pattern=r"Ionization potential",
    )
    core_hole_energy = (
        core_hole_section[0].split("=")[1].split("(")[1].split("e")[0].strip()
    )
    ridgid_shift = core_hole_section[1].split("=")[1].split("e")[0].strip()
    ionization_potential = core_hole_section[2].split("=")[1].split("e")[0].strip()
    tp["energy"]["core_hole_energy"] = float(core_hole_energy)
    tp["energy"]["ridgid_shift"] = float(ridgid_shift)
    tp["energy"]["ionization_potential"] = float(ionization_potential)
    bad_name_data_tree["TP"] = tp

    transitions = extract_transitions(lines)
    # add a section to the data tree for the transition_section
    transition_ds = xr.Dataset(
        {
            "energy": (("transition"), transitions["energy"].to_numpy()),
            "oscillator_strength": (
                ("transition"),
                transitions["oscillator_strength"].to_numpy(),
            ),
            "oslx": (("transition"), transitions["oslx"].to_numpy()),
            "osly": (("transition"), transitions["osly"].to_numpy()),
            "oslz": (("transition"), transitions["oslz"].to_numpy()),
            "osc_r2": (("transition"), transitions["osc_r2"].to_numpy()),
            "<r2>": (("transition"), transitions["<r2>"].to_numpy()),
        },
        coords={
            "transition": transitions["energy"].to_numpy(),
        },
    )

    # add the transition section to the tp data tree
    transition_ds = extract_transitions(lines)
    tp["transitions"] = transition_ds
    if save_checkpoint:
        bad_name_data_tree.to_netcdf(
            Path(file).parent / "tp.nc",
            mode="w",
            engine="h5netcdf",
        )
    return bad_name_data_tree


tp_path = Path.cwd() / "output/C1tp.out"
tp_section = read_stobe_tp(tp_path, "test", "test")
tp_section

<xarray.DataTree>
Group: /
│   Attributes:
│       date:             2025-04-14
│       source:           c:\Users\hduva\.projects\dft-learn\output\C1tp.out
│       description:      EXC file extracted data
│       version:          1.0
│       additional_info:  This dataset contains extracted data from GND files.
│       comment:          test
│       author:           test
├── Group: /GEOMETRY
│       Dimensions:  (atom: 57)
│       Coordinates:
│         * atom     (atom) object 456B 'C01' 'C02' 'C03' 'C04' ... 'N07' 'N08' 'Zn01'
│       Data variables:
│           x        (atom) float64 456B 1.097 0.6501 1.33 0.5761 ... 2.388 1.973 0.0
│           y        (atom) float64 456B -2.796 -4.188 -5.393 ... 2.356 -0.03092 0.0
│           z        (atom) float64 456B -0.1269 -0.1554 -0.3029 ... -0.2149 -0.1647 0.0
│           q        (atom) float64 456B 6.0 4.0 4.0 4.0 4.0 ... 7.0 7.0 7.0 7.0 30.0
│           nuc      (atom) int64 456B 6 6 6 6 6 6 6 6 6 6 6 ... 1 1 7 7 7 7 7 7 7 7 30
│           mass     (atom) float64 456B 12.01 12.01 12.01 12.01 ... 14.01 14.01 65.37
│           neq      (atom) object 456B '1/' '2/' '3/' '4/' ... '54/' '55/' '56/' '57/'
│           grid     (atom) int64 456B 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
│           grp      (atom) int64 456B 32 32 32 32 32 32 32 32 ... 32 32 32 32 32 32 32
├── Group: /BASIS
│       Dimensions:           (atom: 48)
│       Coordinates:
│         * atom              (atom) object 384B 'C01' 'C02' 'C03' ... 'H14' 'H15' 'H16'
│       Data variables:
│           orbital_basis     (atom) object 384B 'O-CARBON iii_iglo' ... 'O-HYDROGEN ...
│           auxiliary_basis   (atom) object 384B 'A-CARBON (5,2;5,2)' ... 'A-HYDROGEN...
│           model_core_basis  (atom) object 384B None 'P-CARBON(+4) (3,1:8,0)' ... None
└── Group: /TP
    ├── Group: /TP/energy
    │       Dimensions:                   ()
    │       Data variables: (12/14)
    │           total_energy              float64 8B -6.622e+04
    │           nuc-nuc_energy            float64 8B 8.772e+04
    │           el-nuc_energy             float64 8B -3.327e+05
    │           kinetic_energy            float64 8B 6.58e+04
    │           coulomb_energy            float64 8B 1.182e+05
    │           ex-cor_energy             float64 8B -5.221e+03
    │           ...                        ...
    │           lumo-energy               float64 8B -4.946
    │           homo-energy               float64 8B -5.143
    │           core-hole-energy          float64 8B -127.2
    │           core_hole_energy          float64 8B -292.0
    │           ridgid_shift              float64 8B 0.0
    │           ionization_potential      float64 8B 292.0
    └── Group: /TP/spin
            Dimensions:           (orbital: 727)
            Coordinates:
              * orbital           (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
            Data variables:
                homo              int64 8B 116
                lumo              int64 8B 117
                alpha_occupation  (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
                alpha_energy      (orbital) float64 6kB -9.41e+03 -1.134e+03 ... 3.588e+03
                alpha_symmetry    (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
                alpha_position    (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
                beta_occupation   (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
                beta_energy       (orbital) float64 6kB -9.41e+03 -1.134e+03 ... 3.575e+03
                beta_symmetry     (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
                beta_position     (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
                core-hole         int64 8B 14

In [ ]:
def read_stobe(
    output_directory: str | Path,
    author: str | None = None,
    comment: str | None = None,
    *,
    save_checkpoint: bool = True,
) -> xr.DataTree:
    """
    Read primary output files and extract relevant data into an xarray DataTree.

    This function extracts geometry, basis sets, energy, and spin information
    from the GND file and organizes it into a structured xarray DataTree.

    Parameters
    ----------
    output_directory : str or Path
        The path to the output directory containing the GND and EXC files.

    author : str
        The name of the author.

    comment : str
        A comment regarding the data extraction.

    save_checkpoint : bool, optional
        Whether to save a checkpoint of the extraction process (default is True).

    Returns
    -------
    xr.Dataset
        A dataset containing the extracted data.
    """
    output_directory = Path(output_directory)
    gnd_path = output_directory / "C1gnd.out"
    exc_path = output_directory / "C1exc.out"
    tp_path = output_directory / "C1tp.out"

    # Read the GND file
    gnd_section = read_stobe_ground(
        gnd_path, author=author, comment=comment, save_checkpoint=save_checkpoint
    )
    # Read the EXC file
    exc_section = read_stobe_excited(
        exc_path, author=author, comment=comment, save_checkpoint=save_checkpoint
    )
    # Read the TP file
    tp_section = read_stobe_tp(
        tp_path, author=author, comment=comment, save_checkpoint=save_checkpoint
    )

    # Combine the sections into a single DataTree
    combined_data_tree = xr.DataTree()
    combined_data_tree["GEOMETRY"] = gnd_section["GEOMETRY"]
    combined_data_tree["BASIS"] = gnd_section["BASIS"]
    combined_data_tree["GND"] = gnd_section["GND"]
    combined_data_tree["EXC"] = exc_section["EXC"]
    combined_data_tree["TP"] = tp_section["TP"]

    # Add metadata to the DataTree
    combined_data_tree.attrs["date"] = datetime.now().strftime("%Y-%m-%d")
    combined_data_tree.attrs["description"] = (
        "Combined data from GND and EXC files extracted data"
    )
    if comment:
        combined_data_tree.attrs["comment"] = comment
    if author:
        combined_data_tree.attrs["author"] = author

    if save_checkpoint:
        combined_data_tree.to_netcdf(
            output_directory / "out.nc",
            mode="w",
            engine="h5netcdf",
        )
    return combined_data_tree


output_directory = Path.cwd() / "output"
combined_data_tree = read_stobe(output_directory, author="test", comment="test")
combined_data_tree

<xarray.DataTree>
Group: /
│   Attributes:
│       date:         2025-04-14
│       description:  Combined data from GND and EXC files extracted data
│       comment:      test
│       author:       test
├── Group: /GEOMETRY
│       Dimensions:  (atom: 57)
│       Coordinates:
│         * atom     (atom) object 456B 'C01' 'C02' 'C03' 'C04' ... 'N07' 'N08' 'Zn01'
│       Data variables:
│           x        (atom) float64 456B 1.097 0.6501 1.33 0.5761 ... 2.388 1.973 0.0
│           y        (atom) float64 456B -2.796 -4.188 -5.393 ... 2.356 -0.03092 0.0
│           z        (atom) float64 456B -0.1269 -0.1554 -0.3029 ... -0.2149 -0.1647 0.0
│           q        (atom) float64 456B 6.0 4.0 4.0 4.0 4.0 ... 7.0 7.0 7.0 7.0 30.0
│           nuc      (atom) int64 456B 6 6 6 6 6 6 6 6 6 6 6 ... 1 1 7 7 7 7 7 7 7 7 30
│           mass     (atom) float64 456B 12.01 12.01 12.01 12.01 ... 14.01 14.01 65.37
│           neq      (atom) object 456B '1/' '2/' '3/' '4/' ... '54/' '55/' '56/' '57/'
│           grid     (atom) int64 456B 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
│           grp      (atom) int64 456B 32 32 32 32 32 32 32 32 ... 32 32 32 32 32 32 32
├── Group: /BASIS
│       Dimensions:           (atom: 48)
│       Coordinates:
│         * atom              (atom) object 384B 'C01' 'C02' 'C03' ... 'H14' 'H15' 'H16'
│       Data variables:
│           orbital_basis     (atom) object 384B 'O-CARBON iii_iglo' ... 'O-HYDROGEN ...
│           auxiliary_basis   (atom) object 384B 'A-CARBON (5,2;5,2)' ... 'A-HYDROGEN...
│           model_core_basis  (atom) object 384B None 'P-CARBON(+4) (3,1:8,0)' ... None
├── Group: /GND
│   ├── Group: /GND/energy
│   │       Dimensions:                   ()
│   │       Data variables:
│   │           total_energy              float64 8B -6.636e+04
│   │           nuc-nuc_energy            float64 8B 8.772e+04
│   │           el-nuc_energy             float64 8B -3.334e+05
│   │           kinetic_energy            float64 8B 6.595e+04
│   │           coulomb_energy            float64 8B 1.186e+05
│   │           ex-cor_energy             float64 8B -5.242e+03
│   │           total_exchange_energy     float64 8B -4.996e+03
│   │           total_correlation_energy  float64 8B -245.9
│   │           lumo-energy               float64 8B -3.365
│   │           homo-energy               float64 8B -3.399
│   └── Group: /GND/spin
│           Dimensions:           (orbital: 727)
│           Coordinates:
│             * orbital           (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
│           Data variables:
│               homo              int64 8B 116
│               lumo              int64 8B 117
│               alpha_occupation  (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
│               alpha_energy      (orbital) float64 6kB -9.408e+03 -1.132e+03 ... 3.612e+03
│               alpha_symmetry    (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
│               alpha_position    (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
│               beta_occupation   (orbital) float64 6kB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
│               beta_energy       (orbital) float64 6kB -9.408e+03 -1.132e+03 ... 3.612e+03
│               beta_symmetry     (orbital) object 6kB '1A' '2A' '3A' ... '726A' '727A'
│               beta_position     (orbital) int64 6kB 1 2 3 4 5 6 ... 723 724 725 726 727
├── Group: /EXC
│   ├── Group: /EXC/energy
│   │       Dimensions:                   ()
│   │       Data variables:
│   │           total_energy              float64 8B -6.607e+04
│   │           nuc-nuc_energy            float64 8B 8.772e+04
│   │           el-nuc_energy             float64 8B -3.327e+05
│   │           kinetic_energy            float64 8B 6.566e+04
│   │           coulomb_energy            float64 8B 1.185e+05
│   │           ex-cor_energy             float64 8B -5.215e+03
│   │           total_exchange_energy     float64 8B -4.969e+03
│   │           total_correla